In [1]:
from __future__ import division

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
%pylab inline

import numpy as np
import pandas as pd
import re
import pprint

import sys
reload(sys)
sys.setdefaultencoding('ISO-8859-1')


import rdflib
from rdflib.graph import Graph, ConjunctiveGraph
from rdflib import Graph, URIRef, BNode, Literal
from rdflib import RDF
from rdflib import Namespace
from rdflib.namespace import OWL, RDF, RDFS, FOAF, XSD
from sys import platform
import slugify
if platform == "linux" or platform == "linux2":
    prefix = "/home/massaro/"
elif platform == "darwin":
    prefix = "/Users/emanuele/"
schema = rdflib.Namespace('http://schema.org/')
time = rdflib.Namespace('http://www.w3.org/2006/time#')

Populating the interactive namespace from numpy and matplotlib


In [2]:
zipCodes = pd.read_csv(prefix+'Dropbox (MIT)/HERUS_Lab/Data/Switzerland/localityPostalCode/zipCodes.csv',
                      encoding = 'utf-8')
canton = pd.read_csv(prefix+'Dropbox (MIT)/HERUS_Lab/Data/Switzerland/canton.csv', header = None)
canton.columns = ["cantonAbbreviation", "cantonName"]

In [4]:
years = [2012, 2013, 2014, 2015, 2016, 2017]
fName = 'Dropbox (MIT)/HERUS_Lab/Data/Switzerland/keyData/keyData%da.csv'%years[5]
data = pd.read_csv(prefix + fName, header = None, encoding = "latin1")

In [11]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,Aeugst am Albis,"1,981",8.6,250,12.8,21.8,62.0,16.2,6.1,2.0,...,18.7,2.1,18.6,30.9,3.5,8.4,2.6,0.2,7.1,4.9
1,Affoltern am Albis,"11,708",5.6,"1,106",27.0,19.7,63.0,17.3,5.8,2.1,...,14.2,4.6,19.1,33.8,5.5,7.4,4.2,0.2,6.2,1.8
2,Bonstetten,"5,326",3.2,717,14.0,24.0,61.2,14.8,4.0,1.9,...,16.5,3.4,20.4,29.1,3.1,11.9,3.8,0.1,6.7,1.9
3,Hausen am Albis,"3,477",3.5,256,13.6,22.2,60.7,17.1,3.5,2.3,...,12.8,2.9,19.4,34.9,2.6,8.7,4.7,0.2,8,1.8
4,Hedingen,"3,660",7.3,560,14.9,22.8,62.3,14.8,4.1,3.6,...,15.8,3.9,22.5,30.1,3.6,9.6,3.8,0.2,6.5,1.8


In [6]:
col = data.ix[:,0]
ac = []
zipC = []
can = []
for i,j in enumerate(col):
    if i == 1939:
        ac.append('VD')
        zipC.append(1083)
        continue
    tmp = zipCodes[zipCodes.municipality == j]
    if len(tmp) > 0:
        ac.append(tmp.cantonAbbreviation[tmp.index[0]])
        zipC.append(tmp.zipCode[tmp.index[0]])
    else: 
        tmp = zipCodes[zipCodes.Gemeindename == j]
        if len(tmp) == 0:
            ac.append('NaN')
            zipC.append('NaN')
            continue
        ac.append(tmp.cantonAbbreviation[tmp.index[0]])
        zipC.append(tmp.zipCode[tmp.index[0]])

In [19]:
g = Graph()
dbo = prefix + "Desktop/tmpOntology/dbpedia_2016-10.owl" 
g.parse(dbo)
dbo = rdflib.Namespace('http://dbpedia.org/ontology/')
res = Namespace('http://dbpedia.org/resource/')

In [20]:
col = data.ix[:,0]
for i,j in enumerate(col):
    nameC = j.replace(" ", "_")
    idS = URIRef(res[nameC])
    g.add((idS, RDF.type, dbo.Town)) 
    g.add((idS, RDF.type, dbo.Municipality))
    g.add((idS, RDF.type, FOAF.name))
    k  = Literal(j)
    g.add((idS, FOAF.Name, k))
    
    
    # canton
    string = str(ac[i])
    tmp = canton[canton.cantonAbbreviation == string]
    if len(tmp) > 0:
        can = tmp.cantonName
        string = can[can.index[0]]
        nameC = string.replace(" ", "_")
        kk = Literal(string)
        c =  URIRef(res[nameC])
        g.add((c, RDF.type, dbo.Canton))
        g.add((idS, dbo.Canton, c))    

        
    # zipCode
    kk = zipC[i]
    kk = Literal(kk)
    zC = URIRef(res[kk])
    g.add((zC, RDF.type, dbo.zipCode))
    g.add((idS, dbo.zipCode, zC)) 
    

    # country
    kk = 'Switzerland'
    kk = Literal(kk)
    nation = URIRef(res[kk])
    g.add((nation, RDF.type, dbo.Country))
    g.add((idS, dbo.Country, nation))  

    
    # population
    kk = data.ix[i,1]
    kk = kk.replace(',', '')
    
    kk = int(kk)
    ns = str(idS)+'population/%d'%kk
    ns = Literal(ns)
    pop =  URIRef(ns)
    g.add((pop, RDF.type, dbo.population))
    g.add((idS, dbo.population, pop))
    
    
    # population density
    kk = data.ix[i,3]
    kk = kk.replace(',', '')
    kk = int(kk)
    ns = str(idS)+'populationdensity/%.2f'%kk
    ns = Literal(ns)
    pop =  URIRef(ns)
    g.add((pop, RDF.type, dbo.population))
    g.add((idS, dbo.populationDensity, pop))

    
    
    # percentOfForeignNationals
    kk = data.ix[i,4]
    kk = float(kk)
    ns = str(idS)+'percentOfForeignNationals/%.2f'%kk
    ns = Literal(ns)
    pop =  URIRef(ns)
    g.add((pop, RDF.type, dbo.population))
    percentOfForeignNationals = URIRef(dbo["percentOfForeignNationals"])
    g.add((idS, dbo.percentOfForeignNationals, pop))
    
    
g.serialize(destination=prefix+'Desktop/outputProva2.nt', format='nt')

In [21]:
g.serialize(destination=prefix+'Desktop/outputProva2.nt', format='nt')